In [180]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
from APIkey import APIkey, APIpw
import json
import csv
import matplotlib.pyplot as plt

In [181]:
#Webscrape https://www.rebellegirls.com/yarnfiber.htm for list of offered brands
url= "https://www.rebellegirls.com/yarnfiber.htm"
page= requests.get(url)

soup=BeautifulSoup(page.content, "html.parser")
results=soup.find(id="accordion")
   
#used ChatGPT to figure out this block of webscraping code
stock=[]
name=[]
brands = soup.find_all('h3')
for brand in brands:
    brand_name = brand.text.strip()
    yarn_list = brand.find_next('div').find_all('li')
    for yarn in yarn_list:
        yarn_name = yarn.text.strip()
        name.append(yarn_name)
        stock.append(brand_name)


In [182]:
#create dataframe of Local Yarn Shop (LYS) offerings
stock_list={"stock":stock, "name":name}
rebelle=pd.DataFrame(stock_list)

rebelle['Rebelle Stock']=rebelle['stock']+ ' '+ rebelle['name']

rebelle=rebelle.drop(['stock', 'name'], axis=1)
rebelle




,Rebelle Stock
0,Adriafil Knitcol
1,Anzula Breeze
2,Anzula Cloud
3,Anzula Haiku
4,Anzula Lunaris
...,...
125,Add-Ins Firestar
126,Add-Ins Angelina
127,Add-Ins Soffsilk
128,Add-Ins Silk Cocoons


In [183]:
def clean(rebelle) -> pd.DataFrame:
    rebelle.columns= rebelle.columns.str.title()
    rebelle.columns= rebelle.columns.str.strip()

    return rebelle

rebelle_list= clean(rebelle)
rebelle_list.iloc[36:61]


,Rebelle Stock
36,Frabjous Fibers Silk Ribbon
37,Galler Yarns Heather Prime Alpaca
38,Hemp for Knitting Allhemp3
39,Hemp for Knitting Allhemp6
40,Hemp for Knitting Hempton
41,Juniper Moon Cumulus
42,Juniper Moon Zooey DK
43,Kelbourne Woolens Mojave
44,Knit Collage Cast Away
45,Knit Collage Pixie Dust


In [184]:
#Code I used to create the ravelry_api.csv file
#Future project is to figure out how to turn this into a FOR LOOP, 
#but couldn't logic it currently and didn't want to waste more time not moving forward on project
#Had to change yarn name in both url and yarn variable, Note for future self:Be sure to CLEAR ALL OUTPUTS after each yarn

#import requests
#import pandas as pd
#import json
#import csv

#url = 'https://api.ravelry.com/patterns/search.json?yarn=Hemp+for+Knitting+Hempton'

#response=requests.get(url, auth= (APIkey, APIpw))

#response

#SEPERATE CELL
#rav_things=response.json()
#patterns= rav_things.get('patterns')
#fiber= pd.DataFrame(response.json()['patterns'])

#fiber['yarn']= 'Hemp for Knitting Hempton'

#fiber.shape

#SEPERATE CELL
#fiber.to_csv('ravelry_api.csv', mode='a', index= False, header= False)



In [185]:
#read in the csv file created from Ravelry API calls
#(data is currently only first page of API results, default is <=100 results)
projects=pd.read_csv('ravelry_api.csv')

In [186]:
#Label each column and see what kind of data we're working with
projects.columns=['free', 'id', 'name', 'permalink', 'personal_attributes', 'first_photo',
       'designer', 'pattern_author', 'pattern_sources', 'yarn']

projects.head

<bound method NDFrame.head of        free       id                                   name  \
0     False   706936                        Puff Stitch Hat   
1      True   650697                   Toe-Up Worsted Socks   
2      True    93685              Child's Self-striping Hat   
3      True   545271                       Best Baby Beanie   
4      True   317207              F193 Baby / Toddler Socks   
...     ...      ...                                    ...   
2823  False   852109                             Persephone   
2824  False  1206148      58 Women Girls WOODLAND Tote & Co   
2825  False  1220650  #61 Girls ZAHA Crop-Cami-Peplum-Dress   
2826  False  1361091                    Clavelia Bucket Bag   
2827   True  7412776             Twelve Point Circular Yoke   

                                 permalink  personal_attributes  \
0                        puff-stitch-hat-9                  NaN   
1                     toe-up-worsted-socks                  NaN   
2           

In [187]:
#Change this column into a string and then expand each key:value pair into own column
projects['pattern_author']= projects['pattern_author'].astype('str')

new = projects["pattern_author"].str.split(",", n=5, expand=True)


#add desired column back to original df
projects['author']= new[4]


In [188]:
def clean(projects) -> pd.DataFrame:
    projects.columns= projects.columns.str.title()
    projects.columns= projects.columns.str.strip()

    return projects

projects= clean(projects)

In [189]:
projects.head(2)

,Free,Id,Name,Permalink,Personal_Attributes,First_Photo,Designer,Pattern_Author,Pattern_Sources,Yarn,Author
0,False,706936,Puff Stitch Hat,puff-stitch-hat-9,NaN,"{'id': 118330077, 'sort_order': 1, 'user_id': ...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","[{'amazon_rating': None, 'amazon_reviews': Non...",Adriafil Knitcol,'name': 'Deborah Kahn'
1,True,650697,Toe-Up Worsted Socks,toe-up-worsted-socks,NaN,"{'id': 56297703, 'sort_order': 1, 'user_id': 1...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","[{'amazon_rating': None, 'amazon_reviews': Non...",Adriafil Knitcol,'name': 'Colette Retrosi'


In [190]:
#clean up column 
projects['Author']=projects['Author'].astype('str')
projects['Author']=projects['Author'].str.strip("'name': '")

projects['Author'].head

<bound method NDFrame.head of 0            Deborah Kah
1        Colette Retrosi
2       Janet D. Russell
3         Danielle Meyer
4         JoAnne Turcott
              ...       
2823     Caitlin ffrench
2824      Adina Bucknall
2825      Adina Bucknall
2826         Craftrebell
2827         Deb Gemmell
Name: Author, Length: 2828, dtype: object>

In [191]:
#drop unwanted columns
projects=projects.drop(['Permalink','Personal_Attributes', 'First_Photo', 'Designer','Pattern_Author', 'Pattern_Sources'], axis=1)

In [192]:
#joined the two datasets together
rebelle_projects= (pd.merge(rebelle, projects, left_on= "Rebelle Stock", right_on= "Yarn"))

rebelle_projects.head(2)

,Rebelle Stock,Free,Id,Name,Yarn,Author
0,Adriafil Knitcol,False,706936,Puff Stitch Hat,Adriafil Knitcol,Deborah Kah
1,Adriafil Knitcol,True,650697,Toe-Up Worsted Socks,Adriafil Knitcol,Colette Retrosi


In [193]:
print(rebelle_projects.isnull().any().any())

False


In [194]:
rebelle_projects.Name.value_counts()

Name
Marcella Evening Gown         4
Inside Cat                    3
Reversible Brioche Cowl       3
Winter Wave Hat               2
Be My Honeybee                2
                             ..
Baby Bobbi Bear               1
Presto Chango                 1
Vanille                       1
Super-Natural Stripes         1
Twelve Point Circular Yoke    1
Name: count, Length: 2516, dtype: int64

In [196]:
rebelle_projects.Author.value_counts()

Author
Taiga Hilliard     58
Lana Hames         34
Shannon Dunbabi    23
Purl Soho          23
Feederbrook Far    22
                   ..
Susan Hickso        1
Isabell Kraemer     1
Dora Stephens       1
Liz Maloney         1
Deb Gemmell         1
Name: count, Length: 1385, dtype: int64

In [195]:
rebelle_projects[rebelle_projects["Author"]=="Taiga Hilliard"]

,Rebelle Stock,Free,Id,Name,Yarn,Author
206,Anzula Cloud,False,637316,Elderberry Wine,Anzula Cloud,Taiga Hilliard
248,Anzula Haiku,False,1299902,Luna Lynn Cardigan,Anzula Haiku,Taiga Hilliard
251,Anzula Haiku,False,836132,Olivia Ann,Anzula Haiku,Taiga Hilliard
287,Anzula Lunaris,False,1248304,Tansy Darling,Anzula Lunaris,Taiga Hilliard
288,Anzula Lunaris,False,1121210,Marian Darling,Anzula Lunaris,Taiga Hilliard
289,Anzula Lunaris,False,1301297,Miss Pebble Dress,Anzula Lunaris,Taiga Hilliard
290,Anzula Lunaris,False,1246073,Zhalna,Anzula Lunaris,Taiga Hilliard
292,Anzula Lunaris,False,7322903,Cibola Cowl,Anzula Lunaris,Taiga Hilliard
294,Anzula Lunaris,False,714927,Camino Cowlette,Anzula Lunaris,Taiga Hilliard
297,Anzula Lunaris,False,748552,Alloy Cowl,Anzula Lunaris,Taiga Hilliard
